<a href="https://colab.research.google.com/github/DaviGomesti/MBA_Machine-Learning/blob/main/Machine_Learning_Criando_modelo_regrecao_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#INSTALAR O SPARK NO COLAB
!pip install pyspark py4j

#IMPORTAR SPARK NO COLAB
from pyspark.sql import SparkSession

#CRIAR SESSÃO SPARK
spark = SparkSession.builder.appName("ExemploPySpark").getOrCreate()

#OBTER CONTEXTO SPARK PARA A SESSÃO
sc = spark.sparkContext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=acfd166336d3e8c5dc653aa24e6972926719b2bb94d0caff789f282befb3d98d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
#IMPORTAR BIBLIOTECAS
from pyspark.ml.regression import LinearRegression, RandomForestRegressor

In [3]:
#IMPORTAR BIBLIOTECAS PARA AVALIAR PERFORMANCE DO MODELO
from pyspark.ml.evaluation import  RegressionEvaluator

In [4]:
#IMPORTAR BIBLIOTECAS HOTENDCODING
from pyspark.ml.feature import VectorAssembler

In [5]:
#INGESTÃO DE DADOS
from pyspark.sql.types import *

In [6]:
#IMPORT DADOS
carros_temp = spark.read.csv('/content/sample_data/Carros.csv', inferSchema=True, header= True, sep=';')
carros_temp.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [7]:
#VAMOS PREVER A POTÊNCIA DO CARRO

In [8]:
#SEPARAR AS COLUNAS DESEJADAS PARA O MODELO
carros = carros_temp.select('Consumo', 'Cilindros', 'Cilindradas', 'HP')
carros.show(5)

+-------+---------+-----------+---+
|Consumo|Cilindros|Cilindradas| HP|
+-------+---------+-----------+---+
|     21|        6|        160|110|
|     21|        6|        160|110|
|    228|        4|        108| 93|
|    214|        6|        258|110|
|    187|        8|        360|175|
+-------+---------+-----------+---+
only showing top 5 rows



In [9]:
#CRIAR O VETOR DE CARACTERISTICAS (VARIÁVEIS INDEPENDENTES)


#DEFINIR AS VARIÁVEIS INDEPENDENTES NO PARAMETRO
veccaracteristicas = VectorAssembler(inputCols=[('Consumo'), ('Cilindros'), ('Cilindradas')], outputCol='caracteristicas')


In [10]:
#AGORA VAMOS APLICAR O OBJETO CRIADO A VARIAVEL CARROS
carros = veccaracteristicas.transform(carros)
carros.show(5)


+-------+---------+-----------+---+-----------------+
|Consumo|Cilindros|Cilindradas| HP|  caracteristicas|
+-------+---------+-----------+---+-----------------+
|     21|        6|        160|110| [21.0,6.0,160.0]|
|     21|        6|        160|110| [21.0,6.0,160.0]|
|    228|        4|        108| 93|[228.0,4.0,108.0]|
|    214|        6|        258|110|[214.0,6.0,258.0]|
|    187|        8|        360|175|[187.0,8.0,360.0]|
+-------+---------+-----------+---+-----------------+
only showing top 5 rows



In [11]:
#DIVIDIR DADOS EM TREINO E TESTE
carrosTreino, carrosTeste = carros.randomSplit([0.7, 0.3])
print(carrosTreino.count())

24


In [12]:
print(carrosTeste.count())



8


In [13]:
#CRIANDO MODELO DE REGRESSÃO
reglin = LinearRegression(featuresCol='caracteristicas', labelCol='HP')
modelo = reglin.fit(carrosTreino)


In [14]:
#PREVISÃO USANDO MODELO CRIADO
previsao = modelo.transform(carrosTeste)
previsao.show(5)

+-------+---------+-----------+---+-----------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|  caracteristicas|        prediction|
+-------+---------+-----------+---+-----------------+------------------+
|     15|        8|        301|335| [15.0,8.0,301.0]|224.60930138613236|
|     21|        6|        160|110| [21.0,6.0,160.0]|182.43173826563276|
|     21|        6|        160|110| [21.0,6.0,160.0]|182.43173826563276|
|     26|        4|       1203| 91|[26.0,4.0,1203.0]|135.29895292340672|
|    133|        8|        350|245|[133.0,8.0,350.0]|195.50601419970434|
+-------+---------+-----------+---+-----------------+------------------+
only showing top 5 rows



In [16]:
#AVALIAR MODELO POR MÉTRICA DE PERFORMANCE
avaliar = RegressionEvaluator(predictionCol='prediction', labelCol='HP', metricName='rmse')
rmse = avaliar.evaluate(previsao)
print(rmse)

64.04330160383925


In [17]:
#GERAR NOVO MODELO COM DADOS DE TESTE
rfreg = RandomForestRegressor(featuresCol='caracteristicas', labelCol='HP')
modelo2 = rfreg.fit(carrosTreino)

In [18]:
previsao2 = modelo2.transform(carrosTeste)
previsao2.show(5)

+-------+---------+-----------+---+-----------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|  caracteristicas|        prediction|
+-------+---------+-----------+---+-----------------+------------------+
|     15|        8|        301|335| [15.0,8.0,301.0]| 178.4126984126984|
|     21|        6|        160|110| [21.0,6.0,160.0]| 146.1781746031746|
|     21|        6|        160|110| [21.0,6.0,160.0]| 146.1781746031746|
|     26|        4|       1203| 91|[26.0,4.0,1203.0]|134.87527777777777|
|    133|        8|        350|245|[133.0,8.0,350.0]|204.07936507936506|
+-------+---------+-----------+---+-----------------+------------------+
only showing top 5 rows



In [19]:
#GERAR UMA MÉTRICA PARA SEGUNDA PREVISÃO
rmse2 = avaliar.evaluate(previsao2)
print(rmse2)

68.80930658951104


In [20]:
print(rmse)

64.04330160383925
